# TP Algorithme de Dijkstra
L'objectif de ce TP est d'implémenter l'algorithme de Dijkstra vu en cours, dans le but de l'appliquer sur un problème concret : calculer le meilleur itinéraire entre deux points d'une ville, et l'afficher sur une carte.

In [ ]:
!pip install osmnx

import networkx as nx
import random
import matplotlib.pyplot as plt
import osmnx as ox

## Exercice 1 : Algorithme de Dijkstra
Ecrivez une fonction `dijkstra(G, s, d)` qui prend un graphe pondéré en entrée, et calcule le plus court chemin du sommet source `s` vers le sommet destination `d`. Votre fonction doit retourner **la longueur de l'itinéraire (en m)** ainsi que **la liste des sommets permettant de reconstituer l'itinéraire**. Testez cette fonction sur un graphe aléatoire.

In [ ]:
def graph_to_weighted(graph):
    """
    Cette fonction transforme un graphe non-pondéré en un graphe pondéré, où chaque arête a un poids aléatoire compris entre 0 et 10.

    :param graph: Le graphe à transformer (networkx.Graph)
    :return: Le graphe pondéré (networkx.Graph)
    """
    weighted_graph = nx.Graph()
    weighted_graph.add_nodes_from(graph.nodes)

    for u, v in graph.edges():
        # attribue le poids à l'arête (u, v)
        weighted_graph.add_edge(u, v, weight=random.randint(0, 10))

    return weighted_graph

W = graph_to_weighted(nx.random_graphs.gnm_random_graph(10, 20))

pos = nx.spring_layout(W)
nx.draw(W, pos, with_labels=True)
# ajoute les poids des arêtes comme étiquettes
edge_labels = nx.get_edge_attributes(W, 'weight')
nx.draw_networkx_edge_labels(W, pos, edge_labels=edge_labels)
plt.show()

In [ ]:
def dijkstra_nx(G, s, d):
    """
    Cette fonction calcule le plus court chemin entre deux sommets d'un graphe pondéré, en utilisant l'implémentation de NetworkX.
    Cette fonction fait office de référence pour la fonction à implémenter.

    :param G: Le graphe à évaluer (networkx.Graph)
    :param s: Le sommet source
    :param d: Le sommet destination
    :return: Le plus court chemin (list), la longueur de l'itinéraire (float)
    """
    return nx.dijkstra_path(G, s, d), nx.dijkstra_path_length(G, s, d)

def min_distance(dist, spt_set, vertices):
    """
    Cette fonction retourne le sommet de distance minimale parmi les sommets non encore traités.

    :param dist: Liste des distances
    :param spt_set: liste des sommets déjà traités
    :param vertices: nombre de sommets
    :return: le sommet de distance minimale
    """
    min_val = float('inf')
    min_index = None

    for v in range(vertices):
        if dist[v] < min_val and not spt_set[v]:
            min_val = dist[v]
            min_index = v

    return min_index

def dijkstra(graph, src, dest):
    """
    Cette fonction calcule le plus court chemin entre deux sommets d'un graphe pondéré, en utilisant l'algorithme de Dijkstra.

    :param graph: Le graphe à évaluer (networkx.Graph)
    :param src: Le sommet source
    :param dest: Le sommet destination
    :return: Le plus court chemin (list), la longueur de l'itinéraire (float)
    """

    # initialisation
    vertices = len(graph)
    dist = [float('inf')] * vertices
    dist[src] = 0
    spt_set = [False] * vertices
    prev = [None] * vertices

    # pour chaque sommet
    for z in range(vertices):
        u = min_distance(dist, spt_set, vertices)
        spt_set[u] = True

        # si on a atteint le sommet destination, on arrête
        if u == dest: break

        # Sinon, pour chaque voisin du sommet
        for v in graph.neighbors(u):

            # si les datas sont un dict
            if isinstance(graph[u][v], dict):
                weight = graph[u][v]['weight']
            else:
                weight = graph[u][v][0]['length']

            # si le voisin n'a pas encore été traité et que la distance est inférieure à la distance actuelle
            if not spt_set[v] and dist[v] > dist[u] + weight:
                dist[v] = dist[u] + weight
                prev[v] = u

    # reconstitution du chemin
    path = []
    current = dest
    while current is not None:
        path.append(current)
        current = prev[current]
    path.reverse()

    return path, dist[dest]

print(dijkstra(W, 0, 9))
print(dijkstra_nx(W, 0, 9))


## Exercice 2 : Application au calcul d'itinéraire
[OSMnx](https://geoffboeing.com/2016/11/osmnx-python-street-networks/) est un package Python permettant d'exploiter au format NetworkX les données disponibles sur [OpenStreetMap](https://openstreetmap.org). Il permet notamment de récupérer toutes les informations du réseau routier, piéton, de transports en commun, etc. et de le manipuler avec NetworkX. De nombreux exemples sont disponibles [sur cette page](https://github.com/gboeing/osmnx-examples).

Google Colaboratory permet d'exécuter des notebooks sur lesquels OSMnx et toutes ses dépendances sont installées.

**Note** : Vous pouvez aussi bien sûr travailler en local, mais il vous faudra installer OSMnx et toutes ses dépendances.

On commence par installer la librairie OSMnx :

*Edit 2023 : la version 1.24 de `numpy` pose problème dans Colab ; on force l'installation de la version 1.23*

In [ ]:
!pip install numpy osmnx folium
!pip install -U scikit-learn

On importe ensuite les paquets nécessaires :

In [ ]:
import networkx as nx
import osmnx as ox
import sklearn as sk
import folium
%matplotlib inline
ox.config(log_console=True)

*A partir de maintenant, toutes les informations, fonctions... sont à chercher dans les documentations fournies au début de cette section.*

**Affichez le réseau routier de Lyon :**

In [ ]:
# récupère le graphe routier de Lyon
lyon_graph = ox.graph_from_place('Lyon, France')

In [ ]:
# affiche le graphe
ox.plot_graph(lyon_graph, edge_linewidth=0.5, edge_color='#999999')

Par défaut, les sommets sont affichés ; quand ils sont trop nombreux, ils nuisent à la lisibilité du graphe. **Affichez de nouveau le réseau routier, en jaune, dans une image de taille 10x10, et sans afficher les sommets.**

In [ ]:
# affiche le réseau routier en jaune dans une image de taille 10x10 et sans afficher les sommets
ox.plot_graph(lyon_graph, edge_color='#FFFF00', figsize=(10,10), node_size=0)

**Combien de sommets et d'arêtes ce graphes comporte-t-il ?**

In [ ]:
# récupère le nombre de nodes
print("Nombre de sommets : ", lyon_graph.number_of_nodes())
# récupère le nombre d'arêtes
print("Nombre d'arêtes : ", lyon_graph.number_of_edges())

**Superposez le réseau routier sur une carte de Lyon** (il existe une fonction d'affichage d'OSMnx pour ça ;)) ; **faites-en sorte que le nom d'une rue soit affiché quand on clique dessus**.

In [ ]:
m = ox.plot_graph_folium(lyon_graph)
m

A présent, l'idée est de déterminer le plus court chemin entre les deux points géographiques fournis, à l'aide de l'algorithme de Dijkstra codé ci-dessus, et de l'afficher sur la carte.

Les numéros des sommets donnés par OpenStreetMap sont ceux de sa base de données *mondiale*, et ne sont donc pas très pratiques si on veut gérer des tableaux de sommets et repérer ces derniers par leur index. **Commencez par *réétiqueter* les sommets, de sortes qu'ils soient numérotés `0, 1, 2...`**

In [ ]:
# ré-étiquetage des sommets du graphe avec l'id original en attribut 'osmid' et en valeur 'id'
conv_nodes_graph = nx.convert_node_labels_to_integers(lyon_graph, first_label=0, ordering='default', label_attribute='osmid')

# affiche les 10 premiers sommets
print(list(conv_nodes_graph.edges(data=True))[:10])

La fonction `nearest_nodes` renvoie le sommet / noeud du graphe le plus proche des coordonnées géographiques (longitude, latitude) données en paramètre. Récupérez les coordonnées des deux points géographiques fournis (via Google Maps, ou [OpenStreetMap](https://www.openstreetmap.org) en faisant un clic-droit puis *Affichez l'adresse*) et affichez les numéros des sommets les plus proches.

In [44]:
#17 : 8 Rue du Garet, 69001 Lyon - 19 Pl. Carnot, 69002 Lyon
# from google maps
# coord1 = (45.766951834267594, 4.836687839133145)
# coord2 = (45.75054937108018, 4.826307225640217)

coord1 = ox.geocode("8 Rue du Garet, 69001 Lyon")
coord2 = ox.geocode("19 Pl. Carnot, 69002 Lyon")

def print_result(coord, node):
    print("Coordonnées : ", coord)
    print("Noeud le plus proche : ", node)

def nearest_nodes_ox(graph, long, lat):
    """
    Récupère le nœud le plus proche des coordonnées géographiques (x, y) dans le graphe G
    à l'aide de la fonction nearest_nodes d'OSMnx
    :param graph: graphe
    :param long: Coordonnées géographiques (longitude)
    :param lat: Coordonnées géographiques (latitude)
    :return: id du nœud le plus proche
    """
    return ox.nearest_nodes(graph, long, lat)

def nearest_nodes(graph, long, lat):
    """
    Récupère le nœud le plus proche des coordonnées géographiques (x, y) dans le graphe G
    :param graph: graphe
    :param long: Coordonnées géographiques (longitude)
    :param lat: Coordonnées géographiques (latitude)
    :return: id du nœud le plus proche
    """
    # récupère les coordonnées des nœuds
    nodes = graph.nodes(data=True)

    # calcule la distance entre les coordonnées et les coordonnées des noeuds
    distances = [((long - node[1]['x'])**2 + (lat - node[1]['y'])**2)**0.5 for node in nodes]

    # récupère l'id du nœud le plus proche
    return distances.index(min(distances))



print("OX :")
node1 = nearest_nodes_ox(conv_nodes_graph, coord1[1], coord1[0])
node2 = nearest_nodes_ox(conv_nodes_graph, coord2[1], coord2[0])
print_result(coord1, node1)
print_result(coord2, node2)

print("Me :")
node1 = nearest_nodes(conv_nodes_graph, coord1[1], coord1[0])
node2 = nearest_nodes(conv_nodes_graph, coord2[1], coord2[0])
print_result(coord1, node1)
print_result(coord2, node2)

OX :
Coordonnées :  (45.7670249, 4.8367981)
Noeud le plus proche :  1781
Coordonnées :  (45.7505164, 4.8263979)
Noeud le plus proche :  20541
Me :
Coordonnées :  (45.7670249, 4.8367981)
Noeud le plus proche :  1781
Coordonnées :  (45.7505164, 4.8263979)
Noeud le plus proche :  20541


Appliquez votre fonction `dijkstra` aux deux sommets du graphe, puis superposez en rouge l'itinéraire obtenu :

In [ ]:
# récupère le chemin le plus court entre les deux sommets
local_path, dist = dijkstra(conv_nodes_graph, node1, node2)

In [ ]:
# affiche le chemin le plus court entre les deux sommets
ox.plot_graph_route(conv_nodes_graph, local_path, route_color='r', route_linewidth=6, node_size=0, figsize=(10, 10))

Appliquez le réétiquetage inverse au chemin obtenu pour retrouver les ID uniques des sommets OpenStreetMap :

In [ ]:
G = lyon_graph.copy()

# récupère les ID des noeuds du chemin
path_ids = [conv_nodes_graph.nodes[node]['osmid'] for node in local_path]

# ajout du path sur le graph lyon_graph
lyon_graph.add_nodes_from(path_ids, color='r')

# affiche le chemin le plus court entre les deux sommets
ox.plot_graph_route(lyon_graph, path_ids, route_color='r', route_linewidth=6, node_size=0, figsize=(10,10))

Enfin, envoyez le chemin obtenu pour l'afficher sur la carte commune, en remplissant votre nom et le nom de la variable content le chemin calculé (liste d'entiers, correspondants aux ID des noeuds OpenStreetMap) :

In [ ]:
#@title Soumission du chemin
Votre_Nom = "Wolo" #@param {type:"string"}
Nom_Variable_Chemin = "path_ids" #@param {type:"string"}

import requests
import urllib.parse
import json

# Soumission du chemin
def submit_path(name, path):
  if not name:
    raise Exception("Le nom ne doit pas être vide")
  if not isinstance(path, list):
    raise Exception("Le chemin doit être une liste")
  if not all(node in G.nodes() for node in path):
      raise Exception("Le chemin doit être une liste de noeuds uniquement")
  if len(path) == 0:
    raise Exception("Le chemin soumis semble vide")
  if len(path) > 1000:
    raise Exception("Le chemin soumis semble trop grand")

  url = "https://3uh5.short.gy/F5Ki4h?name=" + urllib.parse.quote(name) + "&path=" + urllib.parse.quote(json.dumps(path))
  if requests.request("GET", url).status_code == 200:
    print("🎉 Chemin envoyé avec succès !")
  else:
    raise Exception("Echec lors de l'envoi du chemin")

submit_path(Votre_Nom, path_ids)